# Imports

In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------

# Funções

## Carregamento modelo

In [2]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


## Inferencia

In [11]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=200,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

# Executando

In [ ]:
model_name = 'tiiuae/falcon-7b-instruct'
model, peft_config, tokenizer = create_and_prepare_model(model_name)
model = PeftModel.from_pretrained(model, './results/test_falcon_instruct_novoprompt/checkpoint-100')
gen_input = lambda x: f'''
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}\n
###'''               

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.48s/it]


## Inferencias

In [7]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido

###
Prezado cliente, lamentamos muito pelo o que você passou em nosso restaurante. Nós levamos a qualidade dos nossos alimentos muito a sério e ficamos extremamente desapontados em saber que não conseguimos atender às suas expectativas. 

Gostaríamos de pedir desculpas pelo comportamento inadequado dos nossos servidores. Ainda assim, estamos treinando nossa equipe para garantir que isso não aconteça novamente. Nos anos à venir, esperamos ter a oportunidade de servi-lo novamente em nosso restaurante e mostrar que podemos oferecer uma experiência de refeição excepcional. Por favor, não hesite em entrar em contato conosco se precisar de mais informações ou se houver alguma forma que nós podemos melhorar nossa prestação. 

Esperamos ter a oportunidade de servi-lo novamente em nosso restaurante e mostrar que pode

In [8]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!

###
Prezado cliente, lamentamos muito pelo ocorrido em nosso restaurante. Nós levamos a qualidade dos nossos alimentos muito a sério e ficamos extremamente desapontados em saber que o seu experiência não foi satisfatória. Gostaríamos de pedir desculpas pelo atendimento inadequado e pelo tempo de espera prolongado.

Gostaríamos de informar que já tomamos medidas para garantir que isso não aconteça novamente, incluindo treinamento adicional para a nossa equipe e revisão dos nossos processos de qualidade. 
Agradecemos pelo seu feedback e esperamos ter a oportunidade de servi-lo novamente em um futuro próximo, para que possamos mostrar a você a qualidade que sempre buscamos oferecer aos nossos clientes. Por favor, não hesite em entrar em contato conosco se precisar de mais i

In [9]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.

###
Prezado cliente, lamentamos muito pelo o tempo de espera da sua comida em nosso restaurante. Nós levamos a qualidade dos nossos alimentos muito a sério e ficamos extremamente desapontados em saber que a sua experiência não foi satisfatória. Pedimos desculpas pelo atraso na entrega da sua comida e pela falta de qualidade na preparação. 
Gostaríamos de informar que já tomamos medidas para garantir que isso não aconteça novamente, incluindo treinamento adicional para nossa equipe de cozinha e revisão dos nossos processos de preparação dos alimentos. 
Agradecemos pelo seu feedback e esperamos ter a oportunidade de servi-lo novamente em um futuro próximo, para que possamos mostrar a você a qualidade que sempre buscamos oferecer aos nossos clientes. Por favor, não hesite em entrar em conta

In [10]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.

###
Prezado cliente, lamentamos muito pelo ocorrido em nosso restaurante. Sabemos que o tempo de espera na frente dos clientes é muito importante para a nossa empresa e para os nossos clientes.

Gostaríamos de pedir desculpas pelo atraso na entrega dos seus pedidos e pela falta de atendimento na área interna. Nós levamos a qualidade dos nossos alimentos muito a sério e ficamos extremamente desapontados em saber que não conseguimos atender às nossas expectativas durante os fins de semana.

Agradecemos pelo seu feedback e esperamos ter a oportunidade de servi-lo novamente em um futuro próximo, para que possamos mostrar a você a qualidade que sempre buscamos oferecer aos nossos clientes. Por favo